In [1]:
import sys

In [2]:
import time 

In [3]:
import win32com

In [4]:
sys.path.append(r'F:\PycharmProject\PycomCAD')

In [5]:
from pycomcad import *

In [6]:
acad=Autocad()

In [7]:
acad.CurrentFilename

'50-F07281S-S520402.REV.1——无签名.dwg'

In [8]:
acad.IsEarlyBind

True

In [8]:
# acad.TurnOnEarlyBind()

In [40]:
def dotRein(diameter=20,space=200,n=5,scale=50):
    """
    diameter:点钢筋直径
    space:点钢筋间距
    n:一侧的钢筋数
    scale:绘图比例
    """
    blkname='bl'
    space=space/scale
    x1,y1,z1=acad.GetPoint()
    x2,y2,z2=acad.GetPoint()
    x3,y3,z3=acad.GetPoint()
    vectorX=(x2-x1)/((y2-y1)**2+(x2-x1)**2)**0.5
    vectorY=(y2-y1)/((y2-y1)**2+(x2-x1)**2)**0.5
    nvectorX=1
    if vectorY==0:
        nvectorY=1
        nvectorX=0
    else:
        nvectorY=-vectorX/vectorY
    middlePnt=Apoint((x1+x2)/2,(y1+y2)/2)
    mirrorPnt=Apoint((x1+x2)/2+nvectorX,(y1+y2)/2+nvectorY)
    l=acad.AddLine(Apoint(x1,y1),Apoint(x3,y3))
    for i in range(n):
        coord=(x1+i*vectorX*space,y1+i*vectorY*space)
        c=acad.AddCircle(Apoint(*coord),0.5)
        hatch=acad.AddHatch(1,'SOLID',True,(c,))
        c.Mirror(middlePnt,mirrorPnt)
        hatch.Mirror(middlePnt,mirrorPnt)
        ll=l.Copy()
        ll.Move(Apoint(x1,y1),Apoint(*coord))
        lm=ll.Mirror(middlePnt,mirrorPnt)
        if i==0:
            p1=ll.EndPoint
            p2=lm.EndPoint
            acad.AddLine(Apoint(*p1),Apoint(*p2))
    
    

In [42]:
dotRein()

In [57]:
def circleHoleRein(holeDiameter,reD,n=3,respaceFactor=3,radSpace=300,scale=50):
	"""
holeDiameter:Diameter of hole,mm;
respaceFactor:drawn reinforcement's space in the form of onefactor 
reD:Diameter of reinforcement placed around hole,mm;
nh:reinforcement number on one side;
radSpace:interval space of radiate reinforcement,mm;
"""
	import math
	holeD=holeDiameter/scale
	reD=reD/scale
	x,y,z=acad.GetPoint()
	acad.AddCircle(Apoint(x,y),holeD/2)
	c=acad.AddCircle(Apoint(x,y),reD/2+holeD/2)
	pl=acad.AddLwpline(x-holeD,y-holeD/2-reD,x+holeD,y-holeD/2-reD)
	pl.ConstantWidth=1
	pls=[pl]
	for i in range(n):
		plx=pl.Copy()
		plx.Move(Apoint(0,0,0),Apoint(0,-reD*respaceFactor*i,0))
		pls.append(plx)
	for i in range(7):
		rota=(i+1)*math.pi/4
		for pla in pls:
			plb=pla.Copy()
			plb.Rotate(Apoint(x,y,z),rota)
	rpl=acad.AddLwpline(x,y+holeD/2+reD,x,y+3*holeD/4+reD)
	rpl=acad.Handle2Object(rpl.Handle)
	rpl.ConstantWidth=1
	num=int(math.pi*holeD*scale/radSpace)
	print(num)
	rpl.ArrayPolar(num,2*math.pi,Apoint(x,y,z))

In [58]:
circleHoleRein(1200,16,scale=25)

12


In [59]:
circleHoleRein(3000,16,scale=50,respaceFactor=10,radSpace=600)

15


In [60]:
circleHoleRein(600,16,scale=25,respaceFactor=3,radSpace=400)

4


In [13]:
x=acad.GetEntity()

In [16]:
x[0].ArrayPolar(3,3.14,Apoint(*acad.GetPoint()))

(<win32com.gen_py.AutoCAD 2016 Type Library.IAcadLWPolyline instance at 0x1714359984704>,
 <win32com.gen_py.AutoCAD 2016 Type Library.IAcadLWPolyline instance at 0x1714359984928>)